# Import video

Run `pip install opencv-python`

In [21]:

# Open the video file
video_path = "Videos/trappedparticle04.mp4"  # Replace with your video file path

import cv2
from IPython.display import display, clear_output
import time
from PIL import Image

# Open the video file
cap = cv2.VideoCapture(video_path)

# Get the video framerate
fps = cap.get(cv2.CAP_PROP_FPS)
frame_delay = 1 / fps  # Delay between frames in seconds

framecount = 0
while cap.isOpened():
    # print(framecount)
    framecount += 1
    ret, frame = cap.read()
    if not ret:
        break
    
    # Convert BGR to RGB for displaying in Jupyter
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    print(frame_rgb)
    print(frame_rgb.shape)
    break
    img = Image.fromarray(frame_rgb)
    
    # Display the frame
    display(img)
    
    # Pause for the actual frame delay
    time.sleep(frame_delay)
    clear_output(wait=True)

cap.release()
display(f"Video Framerate: {fps} FPS")
display(f"Delay: {frame_delay} seconds")
print("Video playback finished.")
print(f"Total frames: {framecount}")


[[[0 1 0]
  [0 1 0]
  [0 1 0]
  ...
  [0 1 0]
  [0 1 0]
  [0 1 0]]

 [[0 1 0]
  [0 1 0]
  [0 1 0]
  ...
  [0 1 0]
  [0 1 0]
  [0 1 0]]

 [[0 1 0]
  [0 1 0]
  [0 1 0]
  ...
  [0 1 0]
  [0 1 0]
  [0 1 0]]

 ...

 [[0 1 0]
  [0 1 0]
  [0 1 0]
  ...
  [0 1 0]
  [0 1 0]
  [0 1 0]]

 [[0 1 0]
  [0 1 0]
  [0 1 0]
  ...
  [0 1 0]
  [0 1 0]
  [0 1 0]]

 [[0 1 0]
  [0 1 0]
  [0 1 0]
  ...
  [0 1 0]
  [0 1 0]
  [0 1 0]]]
(1922, 2560, 3)


'Video Framerate: 1.4211195077910788 FPS'

'Delay: 0.703670588235294 seconds'

Video playback finished.
Total frames: 1
